In [1]:
# import libraries
import json
import pandas as pd
import re

In [2]:
# read in the rent property scraped json file
path = "../data/raw/property.json"
with open(path, "r") as f:
    property_data = json.load(f)

In [ ]:
# create to store the data
property_df = pd.DataFrame()
rent = []
address =[]
features = []
type = []
furnitured = [] 
pool = []
gym = []
latitude = []
longitude = []

# splite the data in json and store in different list
for i in property_data:
    address.append(property_data[i]["name"])
    rent.append(property_data[i]["cost_text"])
    features. append(property_data[i]["features"])
    type.append(property_data[i]["type"])
    furnitured.append(property_data[i]["furnitured"])
    pool.append(property_data[i]["pool"])
    gym.append(property_data[i]["gym"])
    latitude.append(property_data[i]["coordinates"][0])
    longitude.append(property_data[i]["coordinates"][1])

# consolidate all the data into the same dataframe
property_df["address"] = address
property_df["rent"] = rent
property_df["features"] = features
property_df["type"] = type
property_df["furnitured"] = furnitured
property_df["pool"] = pool
property_df["gym"] = gym
property_df["latitude"] = latitude
property_df["longitude"] = longitude

# check the size of the raw data
property_df.shape

In [5]:
# check and drop duplicate properties
unique_property_df = property_df.drop_duplicates(keep="first").reset_index(drop = True)
unique_property_df.shape

(15004, 9)

In [6]:
# get postcode for all properties from the address
unique_property_df["postcode"] = unique_property_df["address"].\
                                apply(lambda x: int(x[-4:]))

The number of postcode is:  15004


In [10]:
# drop the property with empty features
clean_features = unique_property_df[unique_property_df["features"] != ""].reset_index(drop=True)

# check the size of the data
clean_features.shape

(14970, 10)

In [12]:
num_bed_list = []
num_bath_list = []
num_park_list = []

# split the detail from the property features
for i in clean_features["features"]:

    # find the number of bedroom, bathroom and parking area
    num_bed = re.findall(r'(\d|-|−)\s?Bed', i)
    num_bath = re.findall(r'(\d|-|−)\s?Bath', i)
    num_park = re.findall(r'(\d|-|−)\s?Park', i)

    # if no information, set the number to zero
    if (len(num_bed) == 0) or (num_bed[0].isdigit() == False):
        num_bed = ["0"]
    if (len(num_bath) == 0) or (num_bath[0].isdigit() == False):
        num_bath = ["0"]
    if (len(num_park) == 0) or (num_park[0].isdigit() == False):
        num_park = ["0"]
    
    # append the data together
    num_bed_list.append(int(num_bed[0]))
    num_bath_list.append(int(num_bath[0]))
    num_park_list.append(int(num_park[0]))

# generate the new features of the property
clean_features["num_bed"] = num_bed_list
clean_features["num_bath"] = num_bath_list
clean_features["num_park"] = num_park_list

In [ ]:
# drop the property with no bedroom
clean_features = clean_features[clean_features["num_bed"] != 0].reset_index(drop=True)

In [16]:
# save the cleaned property data with new features
clean_features.to_csv("../data/curated/cleaned_property_with_features.csv", index = None)